In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from idtxl.bivariate_pid import BivariatePID
from idtxl.data import Data

from mesostat.utils.decorators import redirect_stdout
from mesostat.visualization.mpl_colors import base_colors_rgb

# Append base directory
import os,sys
rootname = "conservative-tripartite-testing"
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

import src.null_models_3D as null3D
import src.null_test as nulltest

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/conservative-tripartite-testing


## PID Funictions

In [2]:
decompLabels = ['unq_s1', 'unq_s2', 'shd_s1_s2', 'syn_s1_s2']

In [3]:
def bin_data_1D(data, nBins):
    boundaries = np.quantile(data, np.linspace(0, 1, nBins + 1))
    boundaries[-1] += 1.0E-10
    return np.digitize(data, boundaries, right=False) - 1


def pid_bin(x, y, z, nBins=4):
    dataEff = np.array([
        bin_data_1D(x, nBins),
        bin_data_1D(y, nBins),
        bin_data_1D(z, nBins)
    ])

    return pid(dataEff)


@redirect_stdout
def pid(dataPS):
    settings = {'pid_estimator': 'TartuPID', 'lags_pid': [0, 0]}

    dataIDTxl = Data(dataPS, dim_order='ps', normalise=False)
    pid = BivariatePID()
    rez = pid.analyse_single_target(settings=settings, data=dataIDTxl, target=2, sources=[0, 1])
    rezTrg = rez.get_single_target(2)

    # Getting rid of negative and very low positive PID's.
    # Statistical tests behave unexplectedly - perhaps low values contaminated by roundoff errors?
    return {k : np.clip(rezTrg[k], 1.0E-6, None) for k in decompLabels}

## Models
### Noisy Redundant Scenario

We want to check if white noise added to a purely redundant scenario results in correct identification of redundancy

$$X = T + \nu_X$$
$$Y = T + \nu_Y$$
$$Z = T + \nu_Z$$

where $Y$ is the target of $X$ and $Z$, and

$$T \sim \mathcal{N}(0, 1)$$
$$\nu_X, \nu_Y, \nu_Z \sim \mathcal{N}(0, \sigma)$$

and $\sigma$ is a free parameter, denoting the Noise-To-Signal ratio. So the signal should be a mixture of redundant signal and white noise.

Since the signal is continuous, we bin it using different bin counts.

### Noisy Unique Scenario

Same as before, but

$$X = T + \nu_X$$
$$Y = T + \nu_Y$$
$$Z = \nu_Z$$

### Noisy Redundant Scenario - Discrete Case

It is important to test if false positives are caused by binning, or are an intrinsic property of the noise in the covariate. Here I propose a discretized noisy redundancy model. Instead of added noise, each variable has a random chance to produce the redundant outcome or a purely random outcome.

$$X \sim A_X \nu_X + (1 - A_X) T $$
$$Y \sim A_Y \nu_Y + (1 - A_Y) T $$
$$Z \sim A_Z \nu_Z + (1 - A_Z) T $$

where

$$T, \nu_X, \nu_Y, \nu_Z \sim Ber(0.5) $$
$$A_X \sim Ber(\alpha_X)$$
$$A_Y \sim Ber(\alpha_Y)$$
$$A_Z \sim Ber(\alpha_Z)$$

and $\alpha_X, \alpha_Y, \alpha_Z \in [0, 1]$ are flexible.

So, $\alpha = 0$ means purely redundant signal, and $\alpha=1$ means purely noisy signal.

In [4]:
discrFuncDict = null3D.discr_method_dict()
contFuncDict = null3D.cont_method_dict()

### Testing binning-dependence

In [5]:
# valThrDict = None
valThrDict = {'unq_s1': 0.08, 'unq_s2': 0.08, 'shd_s1_s2': None, 'syn_s1_s2': 0.16}

In [ ]:
taskDict = {
    'yolo':    0.5*np.array([0,0,0]),
    'norand':  0.5*np.array([0,0,1]),
    'randx':   0.5*np.array([1,0,1]),
    'rand':    0.5*np.array([1,1,1])
}

for taskName, params in taskDict.items():
    print(taskName)
    rezDict = {}

    # Do continuous tests
    for funcName, func in contFuncDict.items():
        print('-', funcName)
        
#         for nBins in range(2, 6):
        for nBins in [2]:
            f_data   = lambda: func(10000, *params)
            f_metric = lambda x, y, z: pid_bin(x,y,z, nBins)

            rezDF   = nulltest.run_tests(f_data, f_metric, decompLabels, nTest=100)
            rezDFsh = nulltest.run_tests(f_data, f_metric, decompLabels, nTest=100, haveShuffle=True)

            rezDict[(funcName, nBins)] = (rezDF, rezDFsh)
                        
    # Do discrete tests
    f_metric = lambda x, y, z: pid(np.array([x,y,z]))
    for funcName, func in discrFuncDict.items():
        f_data = lambda: func(10000, *(0.5*params))
        rezDF   = nulltest.run_tests(f_data, f_metric, decompLabels, nTest=100)
        rezDFsh = nulltest.run_tests(f_data, f_metric, decompLabels, nTest=100, haveShuffle=True)

        rezDict[('red_discr', 2)] = (rezDF, rezDFsh)

    for k, v in rezDict.items():
        print(k)
        funcName, nBin = k
        rezDF, rezDFsh = v

        nulltest.plot_test_summary(rezDF, rezDFsh, suptitle=funcName, haveEff=False, valThrDict=valThrDict)
        suffix = '' if valThrDict is None else '_withThr'
        plt.savefig(funcName + '_pid_nbin'+str(nBin)+'_summary_'+taskName+suffix+'.png', dpi=200)
        plt.show()

### Effect of variance

Continuous

In [6]:
nBin = 2
f_metric_cont = lambda x, y, z: pid_bin(x,y,z, nBin)
f_metric_discr = lambda x, y, z: pid(np.array([x,y,z]))

In [ ]:
# Do continuous tests
nData = 10000

alphaStratDict = {
    'PureSrc': lambda alpha: [0,0,alpha],
    'ImpureX': lambda alpha: [alpha,0,alpha],
    'Impure' : lambda alpha: [alpha,alpha,alpha],
}

thrMetricDictDict = {
    'H0_orig' : None,
#     'H0_adj' : {'unq_s1': 0.08, 'unq_s2': 0.08, 'shd_s1_s2': None, 'syn_s1_s2': 0.16}
    'H0_adj' : {'unq_s1': 0.02275646333281051, 'unq_s2': 0.02275646333281051,
                'shd_s1_s2': 0.0004156815533111131, 'syn_s1_s2': 0.15474468603745337}
}

for fName, f_data in contFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        
        f_data_eff = lambda alpha: f_data(nData, *alphaFunc(alpha))
        
        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_param_effect(f_data_eff, f_metric_cont, decompLabels,
                                           nStep=1001, nSkipTest=100, nTest=200, alphaRange=(0, 1),
                                           thrMetricDict=thrMetricDict, plotAlphaSq=False, fontsize=12)

            suffix = 'n_' + str(nData) + '_' + alphaStratName + '_' + h0type

            plt.savefig(fName + '_pid_cont_nBin_'+str(nBin)+'_scatter_vareff_'+suffix+'.svg')
            plt.show()

In [ ]:
nData=10000
for fName, f_data in contFuncDict.items():
    print(fName)
    
    f_data_eff = lambda alpha: f_data(n=nData, sigX=alpha, sigY=alpha, sigZ=alpha)
    nulltest.run_plot_param_effect_test(f_data_eff, f_metric_cont, decompLabels,
                                        nStep=10, nTest=400, alphaRange=(0, 2), valThrDict=valThrDict)
    
    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig(fName + '_pid_nBin2_vareff_n'+str(nData)+suffix+'.png', dpi=200)
    plt.show()

In [ ]:
nData=10000
f_data = lambda alpha: null3D.cont_xor_noisy(n=nData, sigX=alpha, sigY=alpha, sigZ=alpha)
nulltest.run_plot_param_effect_test_single(f_data, f_metric_cont, decompLabels, 0, nTest=400)

Discrete

In [ ]:
# Do discrete tests
nData = 10000

alphaStratDict = {
    'PureSrc': lambda alpha: [0,0,alpha],
    'ImpureX': lambda alpha: [alpha,0,alpha],
    'Impure' : lambda alpha: [alpha,alpha,alpha],
}

thrMetricDictDict = {
    'H0_orig' : None,
    'H0_adj' : {'unq_s1': 0.584222767696213, 'unq_s2': 0.584222767696213, 'shd_s1_s2': None, 'syn_s1_s2': 0.21158892644164767}
}

for fName, f_data in discrFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        
        f_data_eff = lambda alpha: f_data(nData, *alphaFunc(alpha))
        
        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_param_effect(f_data_eff, f_metric_discr, decompLabels,
                                           nStep=1001, nSkipTest=100, nTest=200, alphaRange=(0, 1),
                                           thrMetricDict=thrMetricDict, fontsize=12)

            suffix = 'n_' + str(nData) + '_' + alphaStratName + '_' + h0type

            plt.savefig(fName + '_pid_discr_nBin_'+str(nBin)+'_scatter_vareff_'+suffix+'.svg')
            plt.show()

In [ ]:
nData=10000
for fName, f_data in discrFuncDict.items():
    f_data_eff = lambda alpha: f_data(nData=nData, alphaX=alpha, alphaY=alpha, alphaZ=alpha)
    nulltest.run_plot_param_effect_test(f_data_eff, f_metric_discr, decompLabels,
                                        nStep=10, nTest=400, alphaRange=(0, 1), valThrDict=valThrDict)

    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig(fName + '_pid_vareff_n'+str(nData)+suffix+'.png', dpi=200)
    plt.show()

### Effect of number of samples
Continuous

In [ ]:
alpha=0.25

alphaStratDict = {
    'PureSrc': [0,0,alpha],
    'ImpureX': [alpha,0,alpha],
    'Impure' : [alpha,alpha,alpha],
}

nDataArr = (10**np.linspace(2, 4, 10)).astype(int)
thrLstUnq = [0.2110814493379376, 0.16689265970521838, 0.13685690308881257, 0.11218595017528467, 0.07860180308826488, 0.06210094806194508, 0.0526950184150174, 0.040739636899229006, 0.029856912086735292, 0.02275646333281051]
thrLstRed = [0.04195798086977931, 0.03047641966741261, 0.01652790508850032, 0.012095615389103559, 0.0066037386073174624, 0.003432593102097955, 0.0018820360731625003, 0.001559288121841136, 0.0007872659689172739, 0.0004156815533111131]
thrLstSyn = [0.22514955588715468, 0.21503139471261873, 0.19561667035009186, 0.18203585308711429, 0.172750706469084, 0.16579693515602625, 0.16096080804655502, 0.1590930839954668, 0.1559111008893145, 0.15474468603745337]

thrDictUnq = dict(zip(nDataArr, thrLstUnq))
thrDictRed = dict(zip(nDataArr, thrLstRed))
thrDictSyn = dict(zip(nDataArr, thrLstSyn))

thrMetricDictDict = {
    'H0_orig' : None,
#     'H0_adj' : {'unq_s1': 0.08, 'unq_s2': 0.08, 'shd_s1_s2': None, 'syn_s1_s2': 0.16}
    'H0_adj' : {'unq_s1': thrDictUnq, 'unq_s2': thrDictUnq, 'shd_s1_s2': thrDictRed, 'syn_s1_s2': thrDictSyn}
}


for fName, f_data in contFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        f_data_eff = lambda n: f_data(n, *alphaFunc)

        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_data_effect(f_data_eff, f_metric_cont, decompLabels,
                                          nStep=101, nSkipTest=10, nTest=200, pVal=0.01,
                                          thrMetricDict=thrMetricDict, fontsize=12)

            suffix = 'alpha_' + str(alpha) + '_' + alphaStratName + '_' + h0type

            plt.savefig(fName + '_pid_cont_nBin_'+str(nBin)+'_scatter_nEff_'+suffix+'.svg')
            plt.show()

In [ ]:
alpha=0.5
for fName, f_data in contFuncDict.items():
    print(fName)

    f_data_eff = lambda n: f_data(n=n, aX=alpha, aY=alpha, aZ=alpha)
    nulltest.run_plot_data_effect_test(f_data_eff, f_metric_cont, decompLabels,
                                       nStep=10, nTest=400, valThrDict=valThrDict)
    
    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig(fName + '_pid_nBin2_nEff_sig'+str(sig)+suffix+'.png', dpi=200)
    plt.show()

Discrete

In [ ]:
alpha=0.25

alphaStratDict = {
    'PureSrc': [0,0,alpha],
    'ImpureX': [alpha,0,alpha],
    'Impure' : [alpha,alpha,alpha],
}

nDataArr = (10**np.linspace(2, 4, 10)).astype(int)
thrLstUnq = [0.7574340361396945, 0.720947418015235, 0.727724170531202, 0.8272464705411215, 0.6906704631336307, 0.5872202030236291, 0.5028266540646666, 0.6524915661572469, 0.4263081662581513, 0.584222767696213]
thrLstSyn = [0.2575941657906616, 0.24601433159452815, 0.24779080391562125, 0.22500548226295516, 0.22521972151841443, 0.223708920760494, 0.20122868167962196, 0.2080724784934243, 0.20612904133142615, 0.21158892644164767]

thrDictUnq = dict(zip(nDataArr, thrLstUnq))
thrDictSyn = dict(zip(nDataArr, thrLstSyn))

thrMetricDictDict = {
    'H0_orig' : None,
#     'H0_adj' : {'unq_s1': 0.75, 'unq_s2': 0.75, 'shd_s1_s2': None, 'syn_s1_s2': 0.22}
    'H0_adj' : {'unq_s1': thrDictUnq, 'unq_s2': thrDictUnq, 'shd_s1_s2': None, 'syn_s1_s2': thrDictSyn}
}


for fName, f_data in discrFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        f_data_eff = lambda n: f_data(n, *alphaFunc)

        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_data_effect(f_data_eff, f_metric_discr, decompLabels,
                                          nStep=101, nSkipTest=10, nTest=200, pVal=0.01,
                                          thrMetricDict=thrMetricDict, fontsize=12)

            suffix = 'alpha_' + str(alpha) + '_' + alphaStratName + '_' + h0type

            plt.savefig(fName + '_pid_discr_nBin_'+str(nBin)+'_scatter_nEff_'+suffix+'.svg')
            plt.show()

In [ ]:
alpha=0.5
for fName, f_data in discrFuncDict.items():
    f_data_eff = lambda n: f_data(nData=n, aX=alpha, aY=alpha, aZ=alpha)
    nulltest.run_plot_data_effect_test(f_data_eff, f_metric_discr, decompLabels,
                                       nStep=10, nTest=400, valThrDict=valThrDict)

    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig('redDiscr_pid_nEff_alpha'+str(alpha)+suffix+'.png', dpi=200)
    plt.show()

### Test relationship of synergy and redundancy for fixed data size

#### 1. Finding max synergy parameters - GridSearch3D

In [ ]:
for nData in [1000, 3000, 5000, 7000, 10000]:
    print(nData)
    nulltest.run_gridsearch_3D(null3D.cont_red_noisy, f_metric_cont, 'syn_s1_s2',
                              varLimits=(0, 2), nData=nData, nStep=20)

In [ ]:
for nData in [1000, 3000, 5000, 7000, 10000]:
    print(nData)
    nulltest.run_gridsearch_3D(null3D.discr_red_noisy, f_metric_discr, 'syn_s1_s2',
                              varLimits=(0, 1), nData=nData, nStep=20)

#### 2. Finding max synergy parameters - GridSearch1D

Previous analysis found that in all cases maximal synergy is located at the diagonal $\alpha_x = \alpha_y$

In [7]:
tableauColors = base_colors_rgb(key='tableau')

In [8]:
tableauMap = {
    'unq' : tableauColors[0],
    'red' : tableauColors[2],
    'syn' : tableauColors[3]
}

In [9]:
loopDict = {
    'Cont': [
        ['red', 'unq', 'shd_s1_s2', 'unq_s1',    lambda nData, alpha: null3D.cont_red_noisy(nData, alpha, alpha, alpha)],
        ['red', 'syn', 'shd_s1_s2', 'syn_s1_s2', lambda nData, alpha: null3D.cont_red_noisy(nData, alpha, alpha, 0)],
        ['unq', 'red', 'unq_s1',    'shd_s1_s2', lambda nData, alpha: null3D.cont_unq_noisy(nData, alpha, alpha, alpha)],
        ['unq', 'syn', 'unq_s1',    'syn_s1_s2', lambda nData, alpha: null3D.cont_unq_noisy(nData, alpha, alpha, alpha)],
        ['syn', 'red', 'syn_s1_s2', 'shd_s1_s2', lambda nData, alpha: null3D.cont_xor_noisy(nData, alpha, alpha, alpha)],
        ['syn', 'unq', 'syn_s1_s2', 'unq_s1',    lambda nData, alpha: null3D.cont_xor_noisy(nData, alpha, alpha, alpha)]
    ],
    'Discr' : [
        ['red', 'unq', 'shd_s1_s2', 'unq_s1',    lambda nData, alpha: null3D.discr_red_noisy(nData, alpha, alpha, alpha)],
        ['red', 'syn', 'shd_s1_s2', 'syn_s1_s2', lambda nData, alpha: null3D.discr_red_noisy(nData, alpha, alpha, 0)],
        ['unq', 'red', 'unq_s1',    'shd_s1_s2', lambda nData, alpha: null3D.discr_unq_noisy(nData, alpha, alpha, alpha)],
        ['unq', 'syn', 'unq_s1',    'syn_s1_s2', lambda nData, alpha: null3D.discr_unq_noisy(nData, alpha, alpha, alpha)],
        ['syn', 'red', 'syn_s1_s2', 'shd_s1_s2', lambda nData, alpha: null3D.discr_syn_noisy(nData, alpha, alpha, alpha)],
        ['syn', 'unq', 'syn_s1_s2', 'unq_s1',    lambda nData, alpha: null3D.discr_syn_noisy(nData, alpha, alpha, alpha)]
    ]
}

In [ ]:
nDataLst = (10**np.linspace(2, 4, 10)).astype(int)

for discrKey, loopLst in loopDict.items():
    for labelA, labelB, atomA, atomB, f_data_1D in loopLst:
        prefix = labelA+discrKey+'_pid_1Dscan_'+labelB
        print(prefix)

        alphaMaxLst = []
        thrAdjLst = []
        thrRandLst = []

        for nData in nDataLst:
            print('--', nData)
            alphaMax, thr = nulltest.run_plot_1D_scan(f_data_1D, f_metric_cont, atomA, atomB,
                                                      varLimits=(0, 1), nData=nData, nStep=100, nTest=100,
                                                      colorA = tableauMap[labelA], colorB = tableauMap[labelB])
            plt.savefig(prefix+'_n_'+str(nData)+'.svg')
            plt.show()
            
            # Get also shuffle distribution at this alpha
            datagen_func_noparam = lambda nData: f_data_1D(nData, alphaMax)
            randValues = nulltest.sample_decomp(datagen_func_noparam, f_metric_cont, atomB,
                                                nData=nData, nSample=10000, haveShuffle=True)

            alphaMaxLst += [alphaMax]
            thrAdjLst += [thr]
            thrRandLst += [np.quantile(randValues, 0.99)]

        plt.figure()
#         plt.plot(nDataLst, alphaMaxLst)
        plt.plot(nDataLst, thrAdjLst, label='adjusted', color='purple')
        plt.plot(nDataLst, thrRandLst, label='shuffle')
        plt.legend()
        plt.ylim([0, None])
        plt.savefig(prefix + '_summary.svg')
        plt.show()
        
        print(thrAdjLst)

#### 3. Determining Scatter Relationship

In [ ]:
discrDataMethodDict = {
    'Cont' : null3D.cont_method_dict(),
    'Discr' : null3D.discr_method_dict()
}

atomCombList = {
    ['shd_s1_s2', 'unq_s1'],
    ['shd_s1_s2', 'syn_s1_s2'],
    ['unq_s1',    'shd_s1_s2'],
    ['unq_s1',    'syn_s1_s2'],
    ['syn_s1_s2', 'shd_s1_s2'],
    ['syn_s1_s2', 'unq_s1']
}

In [ ]:
for discrKey, dataMethodsDict in discrDataMethodDict.items():
    for fDataLabel, f_data_3D in dataMethodsDict.items():
        for atomA, atomB in atomCombList:
            nulltest.run_plot_scatter_explore(f_data_3D, f_metric_cont,
                                              atomA, atomB, 3,
                                              varLimits=(0, 1), nData=1000, nTestDim=20)

# Determining testing thresholds for real data

In [10]:
import h5py

In [11]:
# Only test combinations that matter
loopLst = [
    ['red', 'unq', 'shd_s1_s2', 'unq_s1',    lambda nData, alpha: null3D.cont_red_noisy(nData, alpha, alpha, alpha)],
    ['red', 'syn', 'shd_s1_s2', 'syn_s1_s2', lambda nData, alpha: null3D.cont_red_noisy(nData, alpha, alpha, 0)],
    ['unq', 'red', 'unq_s1',    'shd_s1_s2', lambda nData, alpha: null3D.cont_unq_noisy(nData, alpha, alpha, alpha)]
]

# TEX + AUD
nDataLst = [1315, 1209, 3967, 1910, 1724, 4784, 1307, 1324, 5191, 1132, 1014, 3111] + \
           [1070, 510, 2498, 1274, 735, 3407, 1918, 953, 4472, 1008, 630, 2320] + \
           [564, 591, 605, 643, 812, 1040, 1131, 1166, 1263, 1317, 1406, 1412, 1448,
            1525, 1668, 1974, 2438, 2767, 2891, 3228, 3278, 7106, 8209]

In [12]:
for labelA, labelB, atomA, atomB, f_data_1D in loopLst:
    for nData in nDataLst:
        key = labelA + '_' + labelB + '_' + str(nData)
        with h5py.File('pid_rand_dist.h5', 'a') as h5f:
            if key in h5f.keys():
                print(key, 'already done')
                continue
                
        print(key)
        
        randValues = nulltest.run_1D_scan_bare(f_data_1D, f_metric_cont, atomB,
                                               varLimits=(0, 1), nData=nData,
                                               nStep=100, nTest=100, nTestResample=10000)[1]
        
        
        with h5py.File('pid_rand_dist.h5', 'a') as h5f:
            h5f[key] = randValues

red_unq_1315 already done
red_unq_1209 already done
red_unq_3967 already done
red_unq_1910 already done
red_unq_1724 already done
red_unq_4784 already done
red_unq_1307 already done
red_unq_1324 already done
red_unq_5191 already done
red_unq_1132 already done
red_unq_1014 already done
red_unq_3111 already done
red_unq_1070 already done
red_unq_510 already done
red_unq_2498 already done
red_unq_1274 already done
red_unq_735 already done
red_unq_3407 already done
red_unq_1918 already done
red_unq_953 already done
red_unq_4472 already done
red_unq_1008 already done
red_unq_630 already done
red_unq_2320 already done
red_unq_564
red_unq_591
red_unq_605
red_unq_643
red_unq_812
red_unq_1040
red_unq_1131
red_unq_1166
red_unq_1263
red_unq_1317
red_unq_1406
red_unq_1412
red_unq_1448
red_unq_1525
red_unq_1668
red_unq_1974
red_unq_2438
red_unq_2767
red_unq_2891
red_unq_3228
red_unq_3278
red_unq_7106
red_unq_8209
red_syn_1315 already done
red_syn_1209 already done
red_syn_3967 already done
red_syn_